In [ ]:
from __future__ import print_function

import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from linformer import Linformer
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

from module import ViT

In [ ]:
# Training settings
batch_size = 64
epochs = 20
lr = 3e-5
gamma = 0.7
seed = 42

In [ ]:
os.makedirs('data', exist_ok=True)
#创建一个名为 ‘data’ 的目录，如果它已经存在，就忽略这个操作，不要报错。
train_dir = 'data/train'
test_dir = 'data/test'

with zipfile.ZipFile('train.zip') as train_zip:
    #train_zip.extractall('data')#咋感觉这里应该是到train_dir呢。。。
    train_zip.extractall(train_dir)#嗯，一定是大佬写错了
    
with zipfile.ZipFile('test.zip') as test_zip:
    #test_zip.extractall('data')
    test_zip.extractall(test_dir)

train_list = glob.glob(os.path.join(train_dir,'*.jpg'))
#在训练目录 train_dir 中查找所有 .jpg 文件，并将这些文件路径存储在 train_list 中。
test_list = glob.glob(os.path.join(test_dir, '*.jpg'))
#os.path.join 将 train_dir 和 *.jpg 拼接成一个完整路径。
#*.jpg 是一个通配符，表示匹配所有以 .jpg 结尾的文件。
#glob.glob会返回一个列表，包含目录中所有匹配指定模式的文件路径。



In [ ]:
labels = [path.split('/')[-1].split('.')[0] for path in train_list]
#这行代码的作用是从 train_list 中的每个文件路径提取文件名，并且从文件名中提取去掉扩展名后的部分，最后将这些提取到的标签存储到一个名为 labels 的列表中。
#path.split('/')这部分将 path 以 '/'（斜杠）作为分隔符拆分成多个子字符串，返回一个列表。如对于'data/train/image1.jpg'，返回 ['data', 'train', 'image1.jpg']
#[-1] 选取拆分后的最后一个元素，即文件名和扩展名部分。
#split('.')对文件名（如 'image1.jpg'）进行再次拆分,[0]选取拆分后的第一个部分，即文件名去掉扩展名的部分
#最后执行完的labels也是一个列表，比如['image1', 'image2', 'image3']这样的。

下面这段代码可以去掉，顶多就是用于纠错的。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

random_idx = np.random.randint(1, len(train_list), size=9)
fig, axes = plt.subplots(3, 3, figsize=(16, 12))

for idx, ax in enumerate(axes.ravel()):
    img = Image.open(train_list[idx])
    ax.set_title(labels[idx])
    ax.imshow(img)

In [ ]:
train_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),#随机裁剪图像的一部分，并将裁剪后的图像调整为 224x224 的大小
        transforms.RandomHorizontalFlip(),# 随机水平翻转图像（数据增强）
        transforms.ToTensor(),
    ]
)#训练集的图像变换

val_transforms = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),#中心裁剪，裁剪出 224x224 的部分
        transforms.ToTensor(),
    ]
)#验证集的图像变换


test_transforms = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]
)#测试集的图像变换

#我寻思要是自己试试的话，就验证集和测试集通用吧。。。。


In [ ]:
from PIL import Image


class CatsDogsDataset(Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform

    def __len__(self):
        self.filelength = len(self.file_list)
        return self.filelength#返回数据集中图像的总数

    def __getitem__(self, idx):#获取指定索引的图像和标签
        img_path = self.file_list[idx]#根据索引获取图像路径
        img = Image.open(img_path)# 使用 PIL 打开图像
        img_transformed = self.transform(img)#对图像进行变换（如果有的话）

        label = img_path.split("/")[-1].split(".")[0]
        label = 1 if label == "dog" else 0# 如果是"dog"，标签为1，否则为0
        """这个label这看来还得根据数据集改改"""

        return img_transformed, label# 返回变换后的图像和标签

In [ ]:
#每个数据集都通过 CatsDogsDataset 类进行创建
train_data = CatsDogsDataset(train_list, transform=train_transforms)
valid_data = CatsDogsDataset(valid_list, transform=test_transforms)
test_data = CatsDogsDataset(test_list, transform=test_transforms)
#通过 DataLoader 分别创建三个数据加载器
train_loader = DataLoader(dataset = train_data, batch_size=batch_size, shuffle=True )
valid_loader = DataLoader(dataset = valid_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_data, batch_size=batch_size, shuffle=True)

#还是那个想法，感觉自己试试的话也就搞俩试试就行啊
print(len(train_data), len(train_loader))
print(len(valid_data), len(valid_loader))